# Creating Rapid Rotator subsample

August 14, 2022  
Gully & Ryan H.

The goal of this notebook is to make the Rapid Rotator sample.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightkurve as lk
from tqdm import tqdm
import time
import astropy.units as u
import concurrent.futures


sns.set_context('notebook', font_scale=1.5)
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
names = ['EPIC','Campaign','Teff','log g','Prot','ΔProt','hpeak','Rvar','Kp','MG']

In [3]:
df = pd.read_csv('../../data/Reinhold_Hekker2020/table2.dat', 
                 delim_whitespace=True, names=names, na_values='---')

In [4]:
df_rapid_rotator = pd.read_csv('../../data/Rapid_Rotator_Sample.csv')

In [5]:
df_subset = df_rapid_rotator

Looks good!  We see the same trend we had in our proposal figure 2.

## Select a subsample of sources

First search for some high amplitude variable stars

In [ ]:
criterion = (df.Prot < 10)

In [ ]:
criterion.sum()

In [ ]:
#plt.plot(df.Prot, df.Rvar, '.', alpha=0.02);
plt.plot(df.Prot[criterion], df.Rvar[criterion], '.');
#plt.ylim(3e2, 2e5)
plt.xlim(1e0, 1e2)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('$P_{\mathrm{rot}}$')
plt.ylabel('$\propto$ Amplitude (%)')
plt.title('Reinhold & Hekker 2020 Table 2')

In [ ]:
df_subset=df[criterion].reset_index(drop=True)

## Make a subsub sample

### Prepopulate our columns

In [ ]:
df_subset['N_EVEREST'] = np.NaN
df_subset['N_TESS_SPOC'] = np.NaN
df_subset['Period_TESS'] = 0
df_subset['Amplitude_TESS'] = 0
df_subset['Period_K2'] = 0
df_subset['Amplitude_K2'] = 0
df_subset['Sector'] = np.NaN

In [ ]:
df_tiny = df_subset.head(15)

### Remove duplicate entries

In [ ]:
unique_stars = []
duplicate_stars_indices = []
for i in range(len(df_subset)):
    if df_subset.EPIC[i] not in unique_stars:
        unique_stars.append(df_subset.EPIC[i])
    else:
        duplicate_stars_indices.append(i)

In [ ]:
for i in range(len(df_subset) - len(unique_stars)):
    df_subset = df_subset.drop(index=i)

### Predownload so that it runs faster later

Let's find one of the sources that *also* has TESS data available

Delete the cell below if you want to run on the entire subset of 400+ sources...

df_subset = df_tiny

In [ ]:
n_sources = len(df_subset)
n_sources = 10

We want to have at least 1 EVEREST lightcurve and 1 SPOC lightcurve for all sources.

In [ ]:
def add_data(data):
    # data = [mission, index, search result]
    mission, idx, sr = data
    def add_data_helper(mission, idx, sr, num):
        lc = sr[num].download()
        # remove NaNs and normalize the data
        lc = lc.remove_nans().remove_outliers().normalize()
        # find the amplitude percentage
        vector = lc.flux.value
        lo, hi = np.percentile(vector, (5, 95))
        peak_to_valley = hi-lo
        # add the data to the table
        df_subset.loc[idx, f'Amplitude_{mission}'] = peak_to_valley
        # change the lightcurve into a periodogram and find its period
        period = float(lc.to_periodogram(minimum_period=.1, maximum_period=10).period_at_max_power.to_value())
        # add the period to the data table
        df_subset.loc[idx, f'Period_{mission}'] = period
        if mission == 'TESS':
            # find the sector number and add it to the data table
            df_subset.loc[idx, 'Sector'] = lc.sector

    # try using the first available lightcurve
    if len(sr) > 0:
        try:
            add_data_helper(mission, idx, sr, 0)
        except:
            add_data_helper(mission, idx, sr, 1)
        finally:
            return

# download the light curves
def download(data):
    name, index, mission = data
    if mission == 0:
        sr = lk.search_lightcurve(name, mission='TESS')
        df_subset.loc[index, 'N_TESS_SPOC'] = len(sr)
    elif mission == 1:
        sr = lk.search_lightcurve(name, author='EVEREST')
        df_subset.loc[index, 'N_EVEREST'] = len(sr)
    return index, sr

In [ ]:
def main():

    TESS_download = []
    K2_download = []
    for i in range(n_sources):
        # find the name of the star
        name = 'EPIC ' + df_subset.iloc[i].EPIC.astype(int).astype(str)
        TESS_download.append([name, i, 0])
        K2_download.append([name, i, 1])


    TESS_data = []
    K2_data = []

    # use multithreading to make downloading the lightcurves faster
    with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
        TESS_sr = executor.map(download, TESS_download)
        K2_sr = executor.map(download, K2_download)

        for result in TESS_sr:
            TESS_data.append(['TESS', result[0], result[1]])
        for result in K2_sr:
            K2_data.append(['K2', result[0], result[1]])

    # compute the data for each light curve and add it to the new data frame
    for i in range(n_sources):
        add_data(TESS_data[i])
        add_data(K2_data[i])

In [ ]:
if __name__ == '__main__':
    start = time.time()
    main()
    end = time.time()

In [ ]:
end-start

## Complitation times

**15 Stars:**  
Desktop:  
fresh download time ~ 42 seconds  
pre-downloaded time ~ 33.65 seconds  
cached time ~ 2.6 seconds  
  

Laptop:  
fresh download time ~ 46 seconds  
pre-downloaded time ~ 33.8 seconds  
cached time ~ 2.39 seconds  

------------------------------------------------------------  

**416 Stars:**  
Desktop:  
fresh download time ~ 1359.3 seconds ~ 22.6 minutes  
pre-downloaded time ~ 808 seconds ~ 13.5 minutes  
cached time ~ 100 seconds  
  

Laptop:  
fresh download time ~ 1747.5 seconds ~ 29.1 minutes  
pre-downloaded time ~ 936.8 seconds ~ 15.6 minutes  
cached time ~ 66.4 seconds  

------------------------------------------------------------  

**4196 Stars:**
Desktop:  
fresh download time ~ ?? seconds ~ ?? minutes  
pre-downloaded time ~ ?? seconds ~ ?? minutes  
cached time ~ ?? seconds  
  

Laptop:  
fresh download time ~ 15078 seconds ~ 251 minutes ~ 4.19 hours  
pre-downloaded time ~ ?? seconds ~ ?? minutes  
cached time ~ ?? seconds  

## Add more columns to the dataframe

In [ ]:
df_subset['Ratio'] = df_subset['Amplitude_TESS']/df_subset['Amplitude_K2']

In [ ]:
df_subset['T_nearest'] = round(df_subset['Teff'], -2)

In [ ]:
df_subset['T_spot'] = np.NaN

### Find the temperature of the starspot

In [ ]:
columns = [_ for _ in range(2300, 7100, 100)]
index = [_ for _ in range(7000, 2200, -100)]

In [ ]:
one_minus_ratio = pd.read_csv('../../data/one_minus_ratio.csv', names=columns)

In [ ]:
def get_Tspot(T_nearest, Ratio, idx):
    if Ratio < 0.00001:
        df_subset.loc[i, 'T_spot'] = 0
    else:
        intersection = np.argmin(abs(one_minus_ratio[T_nearest] - Ratio))
        T_spot = 7000 - (intersection * 100)
        df_subset.loc[i, 'T_spot'] = T_spot

In [ ]:
for i in range(len(df_subset)):
    get_Tspot(df_subset.loc[i, 'T_nearest'], df_subset.loc[i, 'Ratio'], i)

In [ ]:
df_subset.head(10)

In [ ]:
df_subset.to_csv('../../data/Rapid_Rotator_Sample.csv', index=False)

## Spot check

### Spot check EPIC 212024154

In [ ]:
star = df_subset[df_subset['EPIC'] == 212024154]
star

In [ ]:
T_nearest, Ratio = star.T_nearest.values[0], star.Ratio.values[0]

In [ ]:
ax = plt.subplot()
x = np.linspace(2300, 7000, 48)

# we have to reverse the array since its origin is in the bottom left corner
temps = one_minus_ratio[T_nearest][::-1]

ax.plot(x, temps)

ax.axhline(Ratio, color='red', label='Ratio')
ax.axvline(4400, color='black', linestyle='dashed', label='Approximate intersection')

ax.set_xlabel('Tspot (K)')
ax.set_ylabel('Ratio (%)')
ax.legend(loc='upper left')

In [ ]:
# Find the intersection
intersection = np.argmin(abs(one_minus_ratio[T_nearest] - Ratio))
7000 - (intersection*100)

### Spot check EPIC 228746358

In [ ]:
star = df_subset.loc[df_subset['EPIC'] == 205944272]
star

In [ ]:
T_nearest, Ratio = star.T_nearest.values[0], star.Ratio.values[0]

In [ ]:
ax = plt.subplot()
x = np.linspace(2300, 7000, 48)

# we have to reverse the array since its origin is in the bottom left corner
temps = one_minus_ratio[T_nearest][::-1]

ax.plot(x, temps)

ax.axhline(Ratio, color='red', label='Ratio')
ax.axvline(2550, color='black', linestyle='dashed', label='Approximate intersection')

ax.set_xlabel('Tspot (K)')
ax.set_ylabel('Ratio (%)')
ax.legend(loc='lower right')

In [ ]:
# Find the intersection
intersection = np.argmin(abs(one_minus_ratio[T_nearest] - Ratio))
7000 - (intersection*100)

### Spot check EPIC 220223487

In [ ]:
star = df_subset.loc[df_subset['EPIC'] == 220223487]
star

In [ ]:
T_nearest, Ratio = star.T_nearest.values[0], star.Ratio.values[0]

In [ ]:
ax = plt.subplot()
x = np.linspace(2300, 7000, 48)

# we have to reverse the array since its origin is in the bottom left corner
temps = one_minus_ratio[T_nearest][::-1]

ax.plot(x, temps)

ax.axhline(Ratio, color='red', label='Ratio')
ax.axvline(5400, color='black', linestyle='dashed', label='Approximate intersection')

ax.set_xlabel('Tspot (K)')
ax.set_ylabel('Ratio (%)')
ax.legend(loc='upper left')

In [ ]:
# Find the intersection
intersection = np.argmin(abs(one_minus_ratio[T_nearest] - Ratio))
7000 - (intersection*100)

## Plotting the data

In [ ]:
bad_mask1 = df_subset.Period_TESS > df_subset.Period_K2 * 0.8
bad_mask2 = df_subset.Period_K2 > df_subset.Period_TESS * 0.8
bad_mask3 = df_subset.Period_TESS < 7 
bad_mask4 = df_subset.Period_K2 < 7
mask = bad_mask1 & bad_mask2 & bad_mask3 & bad_mask4

In [ ]:
df_comparison = df_subset[mask].reset_index(drop=True)

In [ ]:
plt.figure(figsize=(6,6))

plt.ylim(0.5, 10)
plt.xlim(0.5, 10)

plt.xlabel('$P_{\mathrm{Kepler}}$')
plt.ylabel('$P_{\mathrm{TESS}}$')

plt.title('Comparison Between TESS and Kepler Amplitudes')

plt.plot(df_comparison.Period_K2, df_comparison.Period_TESS, 'r.', label='Star Amplitude')

x = [0.5, 10]
y = [0.5, 10]
plt.plot(x, y, label='Perfect Correlation')


plt.legend()
plt.show()

In [ ]:
len(df_comparison)

## Receate fig2.pdf plot from proposal

In [ ]:
# create a new dataframe that only includes stars with Kepler and TESS data available
criterion = df_subset.Amplitude_TESS > 0.00005
df_both = df_subset[criterion].reset_index(drop=True)

In [ ]:
TESS_mean = df_both['Amplitude_TESS'].mean()
K2_mean = df_both['Amplitude_K2'].mean()

In [ ]:
ax = plt.subplot(111)

ax.plot(df_both.Period_K2, df_both.Amplitude_K2, '.', color='black', label='Kepler')
ax.plot(df_both.Period_TESS, df_both.Amplitude_TESS, '.', color='red', label='TESS')

fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 20
fig_size[1] = 5
plt.rcParams["figure.figsize"] = fig_size

ax.axhline(TESS_mean, linestyle='dashed', label='TESS mean amplitude', color='green')
ax.axhline(K2_mean, linestyle='dashed', label='Kepler mean amplitude', color='blue')

ax.set_xscale('log')
ax.set_yscale('log')

ax.set_xlim(0.5, 100)
ax.set_ylim(5e-4, 7)

ax.axhline(0.01, linestyle='dotted', label='1% Noise Floor', color='purple')
ax.axvline(27, linestyle='dashed', label='27 days', color='purple')

chartBox = ax.get_position()
ax.set_position([chartBox.x0, chartBox.y0, chartBox.width*0.6, chartBox.height])
ax.legend(loc='upper center', bbox_to_anchor=(1.2, 0.75), shadow=True, ncol=1)

ax.set_xlabel('$P_{\mathrm{rot}}$')
ax.set_ylabel('$\propto$ Amplitude (%)')
ax.set_title('Predicted for 4000 < $T_{\mathrm{eff}}$ < 4500 in TESS')

plt.show()

In [ ]:
ratio = (TESS_mean/K2_mean) * 100
print(f'{ratio} %')

In [ ]:
TESS_mean

In [ ]:
K2_mean

## Recreate Amplitude Comparison Mask

In [ ]:
df_comparison = df_subset#[mask].reset_index(drop=True)

In [ ]:
ax = plt.subplot(111)

ax.plot(df_comparison.Period_K2, df_comparison.Period_TESS, 'r.', label='Star Amplitude')

x = np.linspace(0.5, 10, 10)
y = np.linspace(0.5, 10, 10)

ax.plot(x, y, label='Perfect Correlation', color='green', linestyle='solid')

ax.plot(x, y*0.5, label='Harmonic (one-half)', color='black', linestyle='dashed')

ax.plot(x, y*0.8, label='Maximum Outlier', color='#F29C13', linestyle='dashed', linewidth=4)
ax.plot(x, y*1.2, color='#F29C13', linestyle='dashed', linewidth=4)

x = np.linspace(0.5, 7, 10)

ax.fill_betweenx(x, y*0.57, y*.7, facecolor='green', alpha=0.3)
ax.fill_between(x, y*0.7, y*0.55, facecolor='green', alpha=0.3, label='Great Correlation')

ax.set_ylim(0.5, 10)
ax.set_xlim(0.5, 10)

ax.set_xlabel('$P_{\mathrm{Kepler}}$')
ax.set_ylabel('$P_{\mathrm{TESS}}$')

ax.axvline(7, linestyle='dashed', color='blue')
ax.axhline(7, linestyle='dashed', label='Maximum Period (7 days)', color='blue')

ax.set_title('Comparison Between TESS and Kepler Amplitudes')

chartBox = ax.get_position()
ax.set_position([chartBox.x0, chartBox.y0, chartBox.width*0.5, chartBox.height])
ax.legend(loc='upper center', bbox_to_anchor=(1.3, 0.8), shadow=True, ncol=1)

## Create Ratio plot of TESS/Kepler

In [ ]:
ax = plt.subplot(111)

ax.plot(df_subset.Period_K2, df_subset['Ratio'], 'r.', label='Amplitude ratio of TESS over Kepler')

ax.set_ylim(0, 2.5)
ax.set_xlabel('$P_{\mathrm{rot}}$')
ax.set_ylabel('Ratio')

ax.axhline(1, linestyle='solid', label='Perfect correlation', color='blue')

ax.set_title('Raio of TESS and Kepler Amplitudes')

# chartBox = ax.get_position()
# ax.set_position([chartBox.x0, chartBox.y0, chartBox.width*0.5, chartBox.height])
# ax.legend(loc='upper center', bbox_to_anchor=(1.3, 0.8), shadow=True, ncol=1)
# ax.legend()

In [ ]:
mask = (df_subset.Ratio > 4) | (df_subset.Ratio < 0.001)

In [ ]:
ax = sns.jointplot(data=df_subset[~mask], x="Period_K2", y="Ratio", kind="kde", ylim=(0, 2.5))
ax.ax_joint.axhline(1, color='black', linestyle='dashed')

In [ ]:
df_subset.head()

In [ ]:
plt.hist(df_subset[~mask].Teff)

In [ ]:
# <4000, 4000-5300, >5300
df_subset['Teff_bin'] = np.NaN
for i in range(len(df_subset)):
    temperature = df_subset.loc[i, 'Teff']
    if temperature < 4000:
        Teff_bin = 'Below 4000'
    elif temperature >= 4000 and temperature <= 5300:
        Teff_bin = '4000-5300'
    else:
        Teff_bin = 'Above 5300'
    df_subset.loc[i, 'Teff_bin'] = Teff_bin

In [ ]:
df_subset.head(10)

In [ ]:
plt.hist(df_subset['Teff_bin'])

In [ ]:
ax = sns.jointplot(data=df_subset[~mask], x="Period_K2", y="Ratio", kind="kde", ylim=(0, 2.5))
ax.ax_joint.axhline(1, color='black', linestyle='dashed')
plt.savefig('../../papers/paper1/figures/Amplitude_ratio_kde.png', bbox_inches='tight', dpi=300, facecolor='white', transparent=False)

## Find the duplicate stars in the rapid rotator sample

In [ ]:
unique_stars = []
for i in range(len(df_subset)):
    if df_subset.EPIC[i] not in unique_stars:
        unique_stars.append(df_subset.EPIC[i])

In [ ]:
len(df_subset) - len(unique_stars)

In [ ]:
df_subset[df_subset.EPIC == 211991662]

## See if Eleanor-Lite has been added

In [7]:
for i in range(len(df)):
    name = 'EPIC ' + df_subset.iloc[i].EPIC.astype(int).astype(str)
    search_result = lk.search_lightcurve(name, mission="TESS", author='GSFC-ELEANOR-LITE')
    if len(search_result) != 0:
        print(name)
        break

No data found for target "EPIC 202059204".
No data found for target "EPIC 202059229".
No data found for target "EPIC 202059586".
No data found for target "EPIC 202059600".
No data found for target "EPIC 202059635".
No data found for target "EPIC 202059653".
No data found for target "EPIC 202059766".
No data found for target "EPIC 202060098".
No data found for target "EPIC 202060670".
No data found for target "EPIC 202061819".
No data found for target "EPIC 202064452".
No data found for target "EPIC 202065236".
No data found for target "EPIC 202065998".
No data found for target "EPIC 202066004".
No data found for target "EPIC 202066131".
No data found for target "EPIC 202067687".
No data found for target "EPIC 202067783".
No data found for target "EPIC 202067805".
No data found for target "EPIC 202067808".
No data found for target "EPIC 202067844".
No data found for target "EPIC 202067933".
No data found for target "EPIC 202067952".
No data found for target "EPIC 202068022".
No data fou

KeyboardInterrupt: 